In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/explainable_ai/SDK_Custom_Container_XAI.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/explainable_ai/SDK_Custom_Container_XAI.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/explainable_ai/SDK_Custom_Container_XAI.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本教程将指导您构建一个自定义容器来为Vertex AI Prediction提供一个scikit-learn模型。您将使用FastAPI Python Web服务器框架来创建一个预测和健康端点。您还将为端点启用解释功能。

### 数据集

本教程使用了R.A. Fisher的鸢尾花数据集，这是一个小型数据集，常用于尝试机器学习技术。每个实例都有四个数值特征，这些特征是一种花的不同测量值，还有一个目标标签，标记它是三种鸢尾花中的一种：山鸢尾、变色鸢尾或者维吉尼亚鸢尾。

本教程使用了[scikit-learn库中包含的鸢尾花数据集的副本](https://scikit-learn.org/stable/datasets/index.html#iris-dataset)。

### 目标

目标是：
- 训练一个模型，使用花的测量值作为输入来预测它是哪种类型的鸢尾花。
- 保存模型及其序列化的预处理器。
- 构建一个FastAPI服务器来处理预测和健康检查。
- 构建一个包含模型工件的自定义容器。
- 上传和部署自定义容器到Vertex AI Prediction，并启用可解释性。

本教程更专注于如何使用Vertex AI部署模型，而不是模型本身的设计。

### 成本

本教程使用了Google Cloud的计费组件：

* Vertex AI

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预计使用情况生成成本估算。

### 设置您的本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境满足此笔记本的要求。
您需要以下内容：

* Docker
* Git
* Google Cloud SDK（gcloud）
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南[设置Python开发环境]（https://cloud.google.com/python/setup）和[Jupyter安装指南]（https://jupyter.org/install）提供了满足这些要求的详细说明。以下步骤提供了一组简洁的说明：

1. [安装并初始化Cloud SDK。]（https://cloud.google.com/sdk/docs/）

2. [安装Python 3。]（https://cloud.google.com/python/setup#installing_python）

3. [安装virtualenv]（https://cloud.google.com/python/setup#installing_and_using_virtualenv）并创建使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端shell中的命令行上运行`pip install jupyter`。

5. 要启动Jupyter，请在终端Shell中的命令行上运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

### 安装额外的包

安装额外的包依赖项，这些依赖项没有安装在您的笔记本环境中，例如 NumPy、Scikit-learn、FastAPI、Uvicorn 和 joblib。请使用每个包的最新主要GA版本。

In [ ]:
%%writefile requirements.txt
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev

In [ ]:
# Required in Docker serving container
%pip install -U --user -r requirements.txt

# For local FastAPI development and running
%pip install -U --user "uvicorn[standard]>=0.12.0,<0.14.0" fastapi~=0.63

# Vertex SDK for Python
%pip install -U --user google-cloud-aiplatform

### 重新启动内核

在安装额外的软件包后，您需要重新启动笔记本内核，以便它能够找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在你开始之前

### 设置您的谷歌云项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个账户时，您将获得300美元的免费信用额度，可用于支付计算和存储费用。

1. [确保为您的项目启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您将在本地运行这个笔记本，您将需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保
Cloud SDK 在这个笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 在以 `!` 或 `%` 为前缀的行上运行时，会将其当作 shell 命令，并将 Python 变量插入到这些命令中以`$`或`{}`。

设置您的项目 ID

**如果您不知道您的项目 ID**，您可以使用 `gcloud` 来获得您的项目 ID。

In [ ]:
# Get your Google Cloud project ID from gcloud
shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null

try:
    PROJECT_ID = shell_output[0]
except IndexError:
    PROJECT_ID = None

print("Project ID:", PROJECT_ID)

否则，在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

### 配置项目和资源名称

In [ ]:
MODEL_ARTIFACT_DIR = "custom-container-explainablility-model"  # @param {type:"string"}
REPOSITORY = "custom-container-explainablility"  # @param {type:"string"}
IMAGE = "sklearn-fastapi-server"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "sklearn-explainable-custom-container"  # @param {type:"string"}

`MODEL_ARTIFACT_DIR` - 模型 artifact 的存储桶路径，例如："my-models/fraud-detection/trial-4"

`REPOSITORY` - 要创建或使用的 Artifact Repository 的名称。

`IMAGE` - 要推送的容器映像的名称。

`MODEL_DISPLAY_NAME` - Vertex AI 模型资源的显示名称。

### 创建一个云存储桶

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

在本地训练模型后，您将上传模型构件到一个云存储桶中。使用这个模型构件，然后可以创建 Vertex AI 模型和端点资源，以便提供在线预测和解释。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中保持唯一。

您还可以更改 `REGION` 变量，该变量用于笔记本其余部分的操作。我们建议您 [选择一个 Vertex AI 服务可用的区域](https://cloud.google.com/vertex-ai/docs/general/locations#available_regions)。

In [ ]:
BUCKET_URI = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "[your-region]"  # @param {type:"string"}

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_URI = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

if REGION == "[your-region]":
    REGION = "us-central1"

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查云存储桶中的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

## 使用预处理器训练和存储模型

训练完成后，以下步骤将会将您训练好的模型保存为 joblib（.joblib）文件并上传至云存储。

创建一个目录来存储所有的输出

In [ ]:
%mkdir app

在本地使用鸢尾花数据集训练一个模型，以对花进行分类并返回一个概率。

In [ ]:
%cd app/

import joblib
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression


class IrisClassifier:
    def __init__(self):
        self.X, self.y = load_iris(return_X_y=True)
        self.clf = self.train_model()
        self.iris_type = {0: "setosa", 1: "versicolor", 2: "virginica"}

    def train_model(self) -> LogisticRegression:
        return LogisticRegression(
            solver="lbfgs", max_iter=1000, multi_class="multinomial"
        ).fit(self.X, self.y)

    def predict(self, features: dict):

        X = [
            features["sepal_length"],
            features["sepal_width"],
            features["petal_length"],
            features["petal_width"],
        ]
        prediction = self.clf.predict_proba([X])
        print(prediction)
        return {
            "class": self.iris_type[np.argmax(prediction)],
            "probability": round(max(prediction[0]), 2),
        }


model_local = IrisClassifier()
joblib.dump(model_local, "model.joblib")

%cd ..

在本地测试模型

In [ ]:
model_local.predict(
    features={
        "sepal_length": 4.8,
        "sepal_width": 3,
        "petal_length": 1.4,
        "petal_width": 0.3,
    }
)

在Docker和Vertex AI上创建用于测试预测的实例。

In [ ]:
instances = [
    {"sepal_length": 4.8, "sepal_width": 3, "petal_length": 1.4, "petal_width": 0.3},
    {"sepal_length": 6.2, "sepal_width": 3.4, "petal_length": 5.4, "petal_width": 2.3},
]

创建用于返回预测的分类器代码。

In [ ]:
%%writefile app/classifier.py
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import numpy as np
from fastapi import FastAPI, Request

class IrisClassifier:
    def __init__(self):
        self.X, self.y = load_iris(return_X_y=True)
        self.clf = self.train_model()
        self.iris_type = {
            0: 'setosa',
            1: 'versicolor',
            2: 'virginica'
        }

    def train_model(self) -> LogisticRegression:
        return LogisticRegression(solver='lbfgs',
                                  max_iter=1000,
                                  multi_class='multinomial').fit(self.X, self.y)

    def predict(self, features: dict):
        
        X = [features['sepal_length'], features['sepal_width'], features['petal_length'], features['petal_width']]
        prediction = self.clf.predict_proba([X])
        return {'class': self.iris_type[np.argmax(prediction)], 
                'probability': round(max(prediction[0]), 2)}

In [ ]:
%%writefile classifier.py
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import numpy as np


class IrisClassifier:
    def __init__(self):
        self.X, self.y = load_iris(return_X_y=True)
        self.clf = self.train_model()
        self.iris_type = {
            0: 'setosa',
            1: 'versicolor',
            2: 'virginica'
        }

    def train_model(self) -> LogisticRegression:
        return LogisticRegression(solver='lbfgs',
                                  max_iter=1000,
                                  multi_class='multinomial').fit(self.X, self.y)

    def predict(self, features: dict):
        
        X = [features['sepal_length'], features['sepal_width'], features['petal_length'], features['petal_width']]
        prediction = self.clf.predict_proba([X])
        return {'class': self.iris_type[np.argmax(prediction)], 
                'probability': round(max(prediction[0]), 2)}

在主文件夹和应用文件夹中添加 __init__.py 以启用 .py 文件的导入

In [ ]:
%cd app
with open("__init__.py", "wb") as model_f:
    pass
%cd ..
with open("__init__.py", "wb") as model_f:
    pass

### 上传模型工件和定制代码到云存储

在部署模型进行服务之前，Vertex AI 需要访问云存储中的以下文件：

* `model.joblib`（模型工件）

运行以下命令来上传您的文件：

In [ ]:
!gsutil cp app/model.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

## 构建一个FastAPI服务器

In [ ]:
%%writefile app/main.py
from fastapi import FastAPI, Request
#from starlette.responses import JSONResponse

import joblib
import json
import numpy as np
import pickle
import os

from google.cloud import storage
from classifier import IrisClassifier


app = FastAPI()
'''
gcs_client = storage.Client()

with open("model.joblib", 'wb') as model_f:
    gcs_client.download_blob_to_file(
        f"{os.environ['AIP_STORAGE_URI']}/model.joblib", model_f
    )

#_model = joblib.load("model.joblib")
'''

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()
    print (body)
    
    model = IrisClassifier()
    
    instances = body["instances"]
    output = []
    for i in instances:
        output.append(model.predict(i))
    #return 'class' and 'probability'
    return {"predictions": output}

### 添加预启动脚本
FastAPI将在启动服务器之前执行此脚本。`PORT`环境变量被设置为等于`AIP_HTTP_PORT`，以便在与Vertex AI期望的端口相同的端口上运行FastAPI。

In [ ]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

### 存储测试实例以备后用
要了解有关在JSON中格式化输入实例的更多信息，请[阅读文档。](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details)

In [ ]:
%%writefile instances.json
{
    "instances": [{
        "sepal_length": 4.8,
        "sepal_width": 3,
        "petal_length": 1.4,
        "petal_width": 0.3
    },{
        "sepal_length": 6.2,
        "sepal_width": 3.4,
        "petal_length": 5.4,
        "petal_width": 2.3
    }]
}

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，您的环境已经经过验证。请跳过此步骤。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示进行身份验证来授权您的帐户通过oAuth。

**否则**，请按照以下步骤操作：

1. 在Cloud控制台中，转到[**创建服务帐号密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务帐号**。

3. 在**服务帐号名称**字段中输入一个名称，然后点击**创建**。

4. 在**授予此服务帐号对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中输入"Vertex AI"，选择**Vertex AI管理员**。在过滤框中输入"存储对象管理员"，选择**存储对象管理员**。

5. 点击*创建*。包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中，填入您服务帐号密钥的路径作为`GOOGLE_APPLICATION_CREDENTIALS`变量，然后运行该单元格。

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

可以选择复制您的凭据以在本地运行容器。

In [ ]:
# NOTE: Copy in credentials to run locally, this step can be skipped for deployment
import shutil

GOOGLE_APPLICATION_CREDENTIALS = "[PATH-TO-YOUR-CREDENTIALS.json]"

shutil.copyfile(GOOGLE_APPLICATION_CREDENTIALS, "app/credentials.json")

构建并推送容器到Artifact Registry。

### 建立你的容器

编写Dockerfile，使用`tiangolo/uvicorn-gunicorn-fastapi`作为基础镜像。 这将自动运行FastAPI，并使用Gunicorn和Uvicorn。 访问[FastAPI文档](https://fastapi.tiangolo.com/deployment/docker/) 了解更多关于在Docker中部署FastAPI的信息。

In [ ]:
%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.7

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

构建图像并标记Artifact Registry路径，然后将其推送。

In [ ]:
!docker build \
    --tag={REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE} \
    .

### 本地运行和测试容器（可选）

以分离模式在本地运行容器，并提供容器所需的环境变量。这些环境变量将在部署后由 Vertex Prediction 提供给容器。测试 `/health` 和 `/predict` 路由，然后停止运行中的镜像。

In [ ]:
!docker stop local-iris
!docker rm local-iris
container_id = !docker run -d -p 80:8080 \
    --name=local-iris \
    -e AIP_HTTP_PORT=8080 \
    -e AIP_HEALTH_ROUTE=/health \
    -e AIP_PREDICT_ROUTE=/predict \
    -e AIP_STORAGE_URI={BUCKET_URI}/{MODEL_ARTIFACT_DIR} \
    -e GOOGLE_APPLICATION_CREDENTIALS=credentials.json \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

检查健康路线 (Jiǎnchá jiànkāng lùxiàn)

In [ ]:
!curl localhost/health

检查Docker日志以查找任何问题。

In [ ]:
!docker logs {container_id[0]}

从Docker容器获取一个预测

In [ ]:
!curl -X POST \
  -d @instances.json \
  -H "Content-Type: application/json; charset=utf-8" \
  localhost/predict

停止Docker进程

In [ ]:
!docker stop local-iris

推送容器到制品注册表

创建存储库

In [ ]:
!gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

配置Docker以访问Artifact Registry

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

将您的容器镜像推送到Artifact Registry仓库。

In [ ]:
!docker push {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

## 部署到 Vertex AI

使用 Python SDK 上传和部署您的模型。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

#### 配置解释

##### 在这里，我们将使用Shapley对在表格数据集上训练的模型进行解释。有关更多详细信息，请参阅文档：https://cloud.google.com/vertex-ai/docs/explainable-ai/configuring-explanations#import-model-example

https://cloud.google.com/vertex-ai/docs/explainable-ai/improving-explanations

In [ ]:
XAI = "shapley"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aiplatform.explain.ExplanationParameters(PARAMETERS)

指定输入特征和输出标签名称来配置自定义容器的解释元数据

In [ ]:
EXPLANATION_METADATA = aiplatform.explain.ExplanationMetadata(
    inputs={
        "sepal_length": {},
        "sepal_width": {},
        "petal_length": {},
        "petal_width": {},
    },
    outputs={"probability": {}},
)

上传自定义容器模型

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    explanation_parameters=parameters,
    explanation_metadata=EXPLANATION_METADATA,
)

### 在Vertex AI上部署模型
完成此步骤后，模型将被部署并准备好进行在线预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送预测

使用Python SDK

调用预测端点

In [ ]:
endpoint.predict(instances=instances)

呼叫终端点以获取解释

In [ ]:
endpoint.explain(instances=instances)

### 使用 REST

将一个终端点ID设置为在REST命令中使用

In [ ]:
ENDPOINT_ID = endpoint.name

使用Rest调用预测端点。

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

调用Rest来使用端点进行解释。

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:explain

使用gcloud CLI

调用使用gcloud进行预测的终端点

In [ ]:
!gcloud beta ai endpoints predict $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances.json

使用gcloud调用端点获取解释说明。

In [ ]:
!gcloud beta ai endpoints explain $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances.json

清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}